In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,75.34,86,100,5.79,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,78.12,80,56,5.99,broken clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,28.17,86,100,5.91,light snow
3,3,Longyearbyen,SJ,78.2186,15.6401,8.44,61,20,19.57,few clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,56.82,82,75,27.63,light rain


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Meulaboh,ID,4.1363,96.1285,75.34,86,100,5.79,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,78.12,80,56,5.99,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,56.82,82,75,27.63,light rain
6,6,Kaitangata,NZ,-46.2817,169.8464,54.93,80,93,5.70,overcast clouds
9,9,Mahebourg,MU,-20.4081,57.7000,79.25,89,40,11.50,scattered clouds
10,10,Butaritari,KI,3.0707,172.7902,82.65,73,60,19.48,broken clouds
12,12,Gao,ML,16.6362,1.6370,65.71,12,0,11.83,clear sky
13,13,Isangel,VU,-19.5500,169.2667,80.71,94,100,16.11,overcast clouds
14,14,Busselton,AU,-33.6500,115.3333,65.05,89,20,6.62,few clouds
15,15,Bluff,NZ,-46.6000,168.3333,53.80,69,63,13.04,broken clouds


In [21]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City_ID                422
City                   422
Country                414
Lat                    422
Lng                    422
Max Temp               422
Humidity               422
Cloudiness             422
Wind Speed             422
Current Description    422
dtype: int64


In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                414
City                   414
Country                414
Lat                    414
Lng                    414
Max Temp               414
Humidity               414
Cloudiness             414
Wind Speed             414
Current Description    414
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Meulaboh,ID,75.34,overcast clouds,4.1363,96.1285,NaN
1,Hilo,US,78.12,broken clouds,19.7297,-155.0900,NaN
4,Ushuaia,AR,56.82,light rain,-54.8000,-68.3000,NaN
6,Kaitangata,NZ,54.93,overcast clouds,-46.2817,169.8464,NaN
9,Mahebourg,MU,79.25,scattered clouds,-20.4081,57.7000,NaN
10,Butaritari,KI,82.65,broken clouds,3.0707,172.7902,NaN
12,Gao,ML,65.71,clear sky,16.6362,1.6370,NaN
13,Isangel,VU,80.71,overcast clouds,-19.5500,169.2667,NaN
14,Busselton,AU,65.05,few clouds,-33.6500,115.3333,NaN
15,Bluff,NZ,53.80,broken clouds,-46.6000,168.3333,NaN


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
params["location"] = f"{lat},{lng}"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

# 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()   
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")   

In [25]:
hotel_df.count()

City                   414
Country                414
Max Temp               414
Current Description    414
Lat                    414
Lng                    414
Hotel Name               1
dtype: int64

In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
#Reset display options for dataframe just to make sure that all rows are populated
pd.set_option("display.max_rows", None, "display.max_columns", None)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
718,Morro Bay,US,73.26,broken clouds,35.3658,-120.8499,Inn at Morro Bay


In [27]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
718,Morro Bay,US,73.26,broken clouds,35.3658,-120.8499,Inn at Morro Bay


In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
plt.savefig("../Vacation_Search/WeatherPy_vacation_map.png")

<Figure size 432x288 with 0 Axes>

In [35]:
pwd

'C:\\Users\\cjdas\\World_Weather_Analysis\\Vacation_Search'